In [28]:
import mailparser
import os
import pandas as pd

# Directory to search
directory = 'C:\\Users\\ericb\\Desktop\\Research\\Primary@gmail.com\\Cleaned_Mail\\2023_test\\08\\'
output_csv = 'C:\\Users\\ericb\\Desktop\\Research\\542_Project\\data\\test\\warranted_data_test_output\\mailparser_test_output.csv'

# DataFrame to store the results
df = pd.DataFrame(columns=['filename', 'body', 'subject', 'text_plain', 'text_html', 'text_not_managed', 'defects', 'defects_categories'])

print('Searching files in directory: ' + directory)

# Loop through all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.txt'):  # Check for .eml files
        print('Processing file: ' + filename)

        # Parse the email
        mail = mailparser.parse_from_file(os.path.join(directory, filename))

        # mail = mailparser.parse_from_bytes(file)
        # mail = mailparser.parse_from_file(file)
        # mail = mailparser.parse_from_file_msg(file)
        # mail = mailparser.parse_from_file_obj(file)
        # mail = mailparser.parse_from_string(file)

        # print(mail.attachments) # list of all attachments
        # print(mail.body)

        # print(mail.date) # datetime object in UTC
        # print(mail.defects) # defect RFC not compliance
        # print(mail.defects_categories)  # only defects categories
        # print(mail.delivered_to)
        # print(mail.from_)
        # print(mail.headers) # dict of all headers
        # print(mail.mail) # tokenized mail in a object
        # print(mail.message) # email.message.Message object
        # print(mail.message_as_string) # message as string
        # print(mail.message_id)
        # print(mail.received)
        # print(mail.subject)
        # print(mail.text_plain) # only text plain mail parts in a list
        # print(mail.text_html) # only text html mail parts in a list
        # print(mail.text_not_managed) # all not managed text (check the warning logs to find content subtype)
        # print(mail.to)
        # print(mail.to_domains)
        # print(mail.timezone) # returns the timezone, offset from UTC
        # print(mail.mail_partial) # returns only the mains parts of emails

        # Collect data and append to DataFrame
        df = df.append({
            'filename': filename, 
            'body': mail.body, 
            'subject': mail.subject, 
            'text_plain': mail.text_plain, 
            'text_html': mail.text_html, 
            'text_not_managed': mail.text_not_managed, 
            'defects': str(mail.defects),  # Convert list to string
            'defects_categories': str(mail.defects_categories)  # Convert list to string
        }, ignore_index=True)

# Save the DataFrame to a CSV file
df.to_csv(output_csv, index=False)

print(f"Data saved to {output_csv}")

Email content 'alternative' not handled
Email content 'alternative' not handled
Email content 'alternative' not handled
Email content 'alternative' not handled
Email content 'alternative' not handled
Email content 'alternative' not handled
Email content 'alternative' not handled
Email content 'alternative' not handled
Email content 'alternative' not handled
Email content 'alternative' not handled
Email content 'alternative' not handled


Searching files in directory: C:\Users\ericb\Desktop\Research\Primary@gmail.com\Cleaned_Mail\2023_test\08\
Processing file: _376f0vXRrWR5R6W9rf4NA@geopod-ismtpd-canary-0.txt
Processing file: _5hS-rh6QhW8lvdt7USwww@geopod-ismtpd-35.txt
Processing file: _AmJSCjkTaeQv5tVc0etKQ@geopod-ismtpd-2.txt
Processing file: _dewCW7FTnCWSKjrDppiJw@geopod-ismtpd-5.txt
Processing file: _Dk5VhYJRry4akq_9RLDTA@geopod-ismtpd-4.txt
Processing file: _ebVWls2RUS0Bfa7g9dNUw@geopod-ismtpd-9.txt
Processing file: _eegOk-ASDOKJjfARCrsBQ@geopod-ismtpd-1.txt
Processing file: _gDfSzpjRbmKlpvVGcfEkw@geopod-ismtpd-20.txt
Processing file: _g_cV4dhTqa_lxpOvhPk2Q@geopod-ismtpd-1.txt
Processing file: _HngbeMoTJO5H43mPVjROw@geopod-ismtpd-17.txt
Processing file: _kzBl3-gSna6O0CBD4315Q@geopod-ismtpd-14.txt
Data saved to C:\Users\ericb\Desktop\Research\542_Project\data\test\warranted_data_test_output\mailparser_test_output.csv


In [29]:
# Now that I have the data, I can clean the textual data for use in the model
# print(df['body'][0])
df.head()

,filename,body,subject,text_plain,text_html,text_not_managed,defects,defects_categories
0,_376f0vXRrWR5R6W9rf4NA@geopod-ismtpd-canary-0.txt,"TealNarwhal8831, this FP deal matches your ale...","Home Improvement: 10"" OXO Good Grips Pro Nonst...",[],[],"[TealNarwhal8831, this FP deal matches your al...",[{'multipart/alternative': ['StartBoundaryNotF...,"{'StartBoundaryNotFoundDefect', 'MultipartInva..."
1,_5hS-rh6QhW8lvdt7USwww@geopod-ismtpd-35.txt,Anime awesomeness awaits!\n=E2=80=8C =E2=80=8C...,Welcome to Crunchyroll!,[],[],[Anime awesomeness awaits!\n=E2=80=8C =E2=80=8...,[{'multipart/alternative': ['StartBoundaryNotF...,"{'StartBoundaryNotFoundDefect', 'MultipartInva..."
2,_AmJSCjkTaeQv5tVc0etKQ@geopod-ismtpd-2.txt,\n\n\n\n\n\nAustralian Politics | The Guardian...,Australian politics: Australia news live: US m...,[],[],[\n\n\n\n\n\nAustralian Politics | The Guardia...,[{'multipart/alternative': ['StartBoundaryNotF...,"{'StartBoundaryNotFoundDefect', 'MultipartInva..."
3,_dewCW7FTnCWSKjrDppiJw@geopod-ismtpd-5.txt,\n\n\n\n\n\nThe Best of Guardian Opinion US | ...,The latest op-eds from the Guardian,[],[],[\n\n\n\n\n\nThe Best of Guardian Opinion US |...,[{'multipart/alternative': ['StartBoundaryNotF...,"{'StartBoundaryNotFoundDefect', 'MultipartInva..."
4,_Dk5VhYJRry4akq_9RLDTA@geopod-ismtpd-4.txt,\n\n\n\n\n\nThe Best of Guardian Opinion US | ...,The latest op-eds from the Guardian,[],[],[\n\n\n\n\n\nThe Best of Guardian Opinion US |...,[{'multipart/alternative': ['StartBoundaryNotF...,"{'StartBoundaryNotFoundDefect', 'MultipartInva..."


In [30]:
# Open the CSV file and read it into a DataFrame
df_to_clean = pd.read_csv(output_csv)

In [31]:
import re
import emoji
from bs4 import BeautifulSoup, NavigableString
import quopri
import base64

def replace_emojis(text):
    return emoji.demojize(text, delimiters=("", ""))

def replace_urls_based_on_context(html_content):
    soup = BeautifulSoup(html_content, 'lxml')
    for a_tag in soup.find_all('a'):
        href = a_tag.get('href', '')
        url_type = 'UNSAFE_' if href.startswith('http://') else ''
        if a_tag.img:
            a_tag.string = f'{url_type}IMAGE_URL'
        elif isinstance(a_tag.next, NavigableString) and a_tag.next.strip():
            a_tag.string = f'{url_type}LINK_URL'
        else:
            a_tag.string = f'{url_type}BUTTON_URL'
    return str(soup)

def replace_urls_in_text(text):
    http_url_pattern = re.compile(r'http://\S+')
    text = http_url_pattern.sub('UNSAFE_LINK_URL', text)
    https_url_pattern = re.compile(r'https://\S+')
    text = https_url_pattern.sub('LINK_URL', text)
    return text

def decode_quoted_printable(input_data):
    if isinstance(input_data, bytes):
        return quopri.decodestring(input_data).decode('utf-8', errors='replace')
    else:
        return quopri.decodestring(input_data.encode()).decode('utf-8', errors='replace')

def decode_base64(text):
    return base64.b64decode(text).decode('utf-8', errors='replace')

def clean_text(raw_text):

    #Remove line breaks and continuation equals signs
    raw_text = re.sub(r'=\n', '', raw_text)
    # Decode any quoted-printable text
    raw_text = quopri.decodestring(raw_text.encode()).decode('utf-8', errors='replace')

    # Create a BeautifulSoup object
    soup = BeautifulSoup(raw_text, 'lxml')
    
    # Remove style and script tags and their content
    for tag in soup(['style', 'script', 'img']):
        tag.decompose()

    # Replace URLs in 'a' tags
    for a_tag in soup.find_all('a'):
        href = a_tag.get('href', '')
        url_type = 'UNSAFE ' if href.startswith('http://') else ''
        if a_tag.img:
            a_tag.string = f'{url_type}IMAGE URL'
        elif isinstance(a_tag.next, NavigableString) and a_tag.next.strip():
            a_tag.string = f'{url_type}LINK URL'
        else:
            a_tag.string = f'{url_type}BUTTON URL'

    # Now proceed with extracting text and further cleaning
    text = soup.get_text(separator=' ', strip=True)
    text = replace_emojis(text)

    text = replace_urls_in_text(text)

    # Remove any remaining HTML encoded characters
    text = re.sub(r'&[a-zA-Z0-9#]+;', '', text)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)
    
    # Remove Zero Width Non-Joiner characters
    text = text.replace('\u200b', '')  # Unicode for ZWNBSP
    text = text.replace('\u200c', '')  # Unicode for ZWNJ
    text = text.replace('\u200d', '')  # Unicode for ZWJ
    text = text.replace('\u200e', '')  # Unicode for LEFT-TO-RIGHT MARK
    text = text.replace('\u200f', '')  # Unicode for RIGHT-TO-LEFT MARK


    # Strip string of leading/trailing whitespace
    return text.strip()


In [32]:
# print(df_to_clean['body'][1])

# run the clean_text function on the subject and body columns
df_to_clean['subject'] = df_to_clean['subject'].apply(clean_text)
df_to_clean['body'] = df_to_clean['body'].apply(clean_text)



c:\Users\ericb\anaconda3\envs\machinelearning_20220719\lib\site-packages\bs4\__init__.py:439: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning


In [33]:
df_to_clean.head()


,filename,body,subject,text_plain,text_html,text_not_managed,defects,defects_categories
0,_376f0vXRrWR5R6W9rf4NA@geopod-ismtpd-canary-0.txt,"TealNarwhal8831, this FP deal matches your ale...","Home Improvement: 10"" OXO Good Grips Pro Nonst...",[],[],"['TealNarwhal8831, this FP deal matches your a...",[{'multipart/alternative': ['StartBoundaryNotF...,"{'StartBoundaryNotFoundDefect', 'MultipartInva..."
1,_5hS-rh6QhW8lvdt7USwww@geopod-ismtpd-35.txt,Anime awesomeness awaits! ...,Welcome to Crunchyroll!,[],[],['Anime awesomeness awaits!\n=E2=80=8C =E2=80=...,[{'multipart/alternative': ['StartBoundaryNotF...,"{'StartBoundaryNotFoundDefect', 'MultipartInva..."
2,_AmJSCjkTaeQv5tVc0etKQ@geopod-ismtpd-2.txt,Australian Politics | The Guardian LINK_URL La...,Australian politics: Australia news live: US m...,[],[],['\n\n\n\n\n\nAustralian Politics | The Guardi...,[{'multipart/alternative': ['StartBoundaryNotF...,"{'StartBoundaryNotFoundDefect', 'MultipartInva..."
3,_dewCW7FTnCWSKjrDppiJw@geopod-ismtpd-5.txt,The Best of Guardian Opinion US | The Guardian...,The latest op-eds from the Guardian,[],[],['\n\n\n\n\n\nThe Best of Guardian Opinion US ...,[{'multipart/alternative': ['StartBoundaryNotF...,"{'StartBoundaryNotFoundDefect', 'MultipartInva..."
4,_Dk5VhYJRry4akq_9RLDTA@geopod-ismtpd-4.txt,The Best of Guardian Opinion US | The Guardian...,The latest op-eds from the Guardian,[],[],['\n\n\n\n\n\nThe Best of Guardian Opinion US ...,[{'multipart/alternative': ['StartBoundaryNotF...,"{'StartBoundaryNotFoundDefect', 'MultipartInva..."


In [36]:
print(df_to_clean['body'][1])

# save df_to_clean['body'][i] to txt file
for i in range (0, len(df_to_clean['body'])):
    with open('C:\\Users\\ericb\\Desktop\\Research\\542_Project\\data\\test\\warranted_data_test_output\\mailparser_test_output_body' + str(i) + '.txt', 'w') as f:
        f.write(df_to_clean['body'][i])

# save df_to_clean['text_not_managed'][i] to txt file
for i in range(0, len(df_to_clean['text_not_managed'])):
    with open('C:\\Users\\ericb\\Desktop\\Research\\542_Project\\data\\test\\warranted_data_test_output\\mailparser_test_output_text_not_managed' + str(i) + '.txt', 'w') as f:
        f.write(df_to_clean['text_not_managed'][i])


Anime awesomeness awaits!                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           Welcome to the internet's premier anime experience! Get started by browsing our always growing library and filling your queue. Then just sit back and enjoy all the anime you can handle! Browse Now ( LINK_URL ) fb ( LINK_URL ) tw ( LINK_URL ) yt ( LINK_URL ) insta ( LINK_URL ) crunchyroll ( LINK_URL ) Crunchyroll PO BOX 405